# Import Required Libraries

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2

from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Input, MaxPooling2D, UpSampling2D
from tensorflow.keras.optimizers import Adam

# Load Data

In [ ]:
root_dir = "/mnt/hdd/Datasets/medical_image/"

In [ ]:
image_paths = os.listdir(root_dir)
len(image_paths)

In [ ]:
images = []

for image_path in image_paths:
    img = load_img(root_dir + image_path, target_size=(64, 64), color_mode="grayscale")
    img = img_to_array(img)
    img = img / 255.
    images.append(img)

In [ ]:
images = np.array(images)

In [ ]:
type(images)

In [ ]:
def visualize_df(df: np.ndarray):
    fig, axes = plt.subplots(4, 4, figsize=(12, 12))
    for i, ax in enumerate(axes.ravel()):
        if i < len(df):
            img = df[i]
            ax.imshow(img, cmap="gray")
            ax.axis("off")
        else:
            ax.axis("off")

    plt.tight_layout()
    plt.show()

In [ ]:
visualize_df(images)

In [ ]:
noise_factor = 0.2

noise_images = []

for image in images:
    noisy = image + noise_factor * np.random.randn(*image.shape)
    noisy = np.clip(noisy, 0., 1.)
    noise_images.append(noisy)

In [ ]:
noise_images = np.array(noise_images)

In [ ]:
plt.imshow(images[0], cmap="gray")

In [ ]:
plt.imshow(noise_images[0], cmap="gray")

In [ ]:
noise_train = noise_images[:100]
noise_test = noise_images[100:]

noise_train.shape, noise_test.shape

In [ ]:
normal_train = images[:100]
normal_test = images[100:]

normal_train.shape, normal_test.shape

# Model

In [ ]:
autoencoder1 = Sequential([
    # Input
    Input((64, 64, 1)),
    
    # Encoder
    Conv2D(32, (3, 3), strides=2, padding="same"),
    Conv2D(16, (3, 3), strides=2, padding="same", activation='relu'),
    
    Conv2D(16, (3, 3), strides=1, padding="same",activation='relu'),
    
    # Decoder
    Conv2DTranspose(64, (3, 3), strides=2, padding="same", activation='relu'),
    Conv2DTranspose(1, (3, 3), strides=2, activation='sigmoid', padding="same"),
])

In [ ]:
autoencoder1.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001))
autoencoder1.summary()

In [ ]:
autoencoder1.fit(noise_train,        
                 normal_train,
                 validation_data=(noise_test, normal_test),
                 epochs=16, 
                 batch_size=128)

In [ ]:
evaluation = autoencoder1.evaluate(noise_test, noise_test)
evaluation

In [ ]:
predicted = autoencoder1.predict(noise_test)

In [ ]:
def plot_predictions(test, preds):
    plt.figure(figsize=(12, 12))
    
    # Noise test
    for i in range(16):
        plt.subplot(4, 8, i + 1)
        plt.imshow(test[i], cmap='gray')
        plt.axis('off')
        plt.title('Noised')
    
    # Predicted
    for i in range(16):
        plt.subplot(4, 8, i + 17)
        plt.imshow(preds[i], cmap='gray')
        plt.axis('off')
        plt.title('Predicted')
    
    plt.show()

In [ ]:
plot_predictions(noise_test, predicted)

In [ ]:
input_layer = Input(shape=(64, 64, 1))

# Encoder
x = Conv2D(32, (3, 3), activation="relu", padding="same")(input_layer)
x = MaxPooling2D((2, 2), padding="same")(x)
x = Conv2D(64, (3, 3), activation="relu", padding="same")(x)
encoded = MaxPooling2D((2, 2), padding="same")(x)

# Decoder
x = Conv2D(64, (3, 3), activation="relu", padding="same")(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation="relu", padding="same")(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation="sigmoid", padding="same")(x)

autoencoder2 = Model(input_layer, decoded)

In [ ]:
autoencoder2.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
autoencoder2.fit(noise_train, noise_train, epochs=50, batch_size=128, shuffle=True, validation_data=(noise_test, noise_test))

In [ ]:
predicted = autoencoder2.predict(noise_test)

In [ ]:
plot_predictions(noise_test, predicted)